Python Implementation of: https://www.desmos.com/calculator/nw4zrqrftx 

In [24]:
import param as pm
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv
import math
# pn.config.throttled = True
# pn.extension()

In [25]:
class Uniswap(pm.Parameterized):
    L = pm.Number(73.5, bounds=(0.05, 400), step=0.1)
    k = pm.Number(precedence=-1)
    
    x_start = pm.Number(precedence=-1)
    x_end = pm.Number(167, bounds=(0.05,302.7))
    x_low = pm.Number(precedence=-1)
    x_high = pm.Number(precedence=-1)
    
    y_start = pm.Number(188, bounds=(0,226))
    y_end = pm.Number(precedence=-1)
    y_low = pm.Number(precedence=-1)
    y_high = pm.Number(precedence=-1)
    
    xmax = pm.Number(400, precedence=-1)
    ymax = pm.Number(400, precedence=-1)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.update()
    
    def yx(self, x):
        return self.L**2 / x
    
    def xy(self, y):
        return self.L**2 / y
    
    def p(self, x):
        return self.L**2 / x**2
    
    def truncate(self, x):
        return math.floor(x * 1000) / 1000
    
    @pn.depends('L', 'x_end', 'y_start', watch=True)
    def update(self):
        self.k = self.L**2
        
        self.x_start = self.xy(self.y_start)
        self.x_end = self.x_end
        self.y_start = self.y_start
        self.y_end = self.yx(self.x_end)
        self.x_low = min(self.x_start,self.x_end)
        self.x_high = max(self.x_start,self.x_end)
        self.y_low = min(self.y_start,self.y_end)
        self.y_high = max(self.y_start,self.y_end)
        
        self.p_start = self.p(self.x_start)
        self.p_end = self.p(self.x_end)
        self.p_execution = - (self.yx(self.x_end) - self.yx(self.x_start)) / (self.x_end - self.x_start)
        self.p_execution_truncated = self.truncate(self.p_execution)
        self.p_starttruncated = self.truncate(self.p_start)
        self.p_endtruncated = self.truncate(self.p_end)
        
        self.p_angle = -np.arctan(self.p_execution)
        self.p_startangle = -np.arctan(self.p_start)
        self.p_endangle = -np.arctan(self.p_end)
        
        self.x_in = self.truncate(self.x_end - self.x_start)
        self.x_out = self.truncate(self.x_start - self.x_end)
        
        self.y_in = self.truncate(self.y_end - self.y_start)
        self.y_out = self.truncate(self.y_start - self.y_end)
    
    def view_ll(self):
        point = hv.Points((self.L, self.L)).opts(color='purple',size=10, xlim=(0,self.xmax), ylim=(0,self.ymax))
        return point
    
    def view_x_start_y_start(self):
        point = hv.Points((self.x_start, self.y_start)).opts(color='green',size=10, xlim=(0,self.xmax), ylim=(0,self.ymax))
        return point
    
    def view_x_end_y_start(self):
        point = hv.Points((self.x_end, self.y_start)).opts(color='black',size=10, xlim=(0,self.xmax), ylim=(0,self.ymax))
        return point
    
    def view_end_price(self):
        point = hv.Points((self.x_end, self.y_end)).opts(color='orange',size=10, xlim=(0,self.xmax), ylim=(0,self.ymax))
        return point
    
    def view_points(self):
        points = self.view_ll() * self.view_x_start_y_start() * self.view_x_end_y_start() * self.view_end_price()
        return points
    
    def xy_curve(self):
        xs = np.linspace(*self.param['x_end'].bounds)
        ys = self.yx(xs)
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
    
    def x_in_curve(self):
        xs = np.linspace(self.x_low, self.x_high)
        ys = [self.y_start] * len(xs)
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
    
    def y_out_curve(self):
        xs = np.linspace(self.x_end, self.x_end)
        ys = np.linspace(self.y_low, self.y_high)
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
    
    def execution_curve(self):
        xs = np.linspace(self.x_low, self.x_high)
        ys = self.y_start + self.y_end - self.p_execution*xs
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
    
    def start_price_curve(self):
        xs = np.linspace(*self.param['x_end'].bounds)
        ys = -self.p_start * xs + 2*self.y_start
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
    
    def end_price_curve(self):
        xs = np.linspace(*self.param['x_end'].bounds)
        ys = -self.p_end * xs + 2*self.y_end
        df = pd.DataFrame({'x':xs,'y':ys})
        return df
        
    def view_xy_curve(self):
        return self.xy_curve().hvplot.line(x='x',y='y', xlim=(0,self.xmax), ylim=(0,self.ymax), color='purple')
    
    def view_x_in_curve(self):
        return self.x_in_curve().hvplot.line(x='x',y='y', xlim=(0,self.xmax), ylim=(0,self.ymax), color='skyblue', line_dash='dashed')
    
    def view_y_out_curve(self):
        return self.y_out_curve().hvplot.line(x='x',y='y', xlim=(0,self.xmax), ylim=(0,self.ymax), color='red', line_dash='dashed')
    
    def view_execution_curve(self):
        return self.execution_curve().hvplot.line(x='x',y='y', xlim=(0,self.xmax), ylim=(0,self.ymax), color='black', line_dash='dashed')
    
    def view_start_price_curve(self):
        return self.start_price_curve().hvplot.line(x='x',y='y', xlim=(0,self.xmax), ylim=(0,self.ymax), color='green', line_width=2, line_dash='dashed', alpha=0.5)
    
    def view_end_price_curve(self):
        return self.end_price_curve().hvplot.line(x='x',y='y', xlim=(0,self.xmax), ylim=(0,self.ymax), color='orange', line_width=2, line_dash='dashed', alpha=0.5)
        
    def view_curves(self):
        chart = self.view_xy_curve() * self.view_x_in_curve() * self.view_y_out_curve() * self.view_execution_curve() * self.view_start_price_curve() * self.view_end_price_curve()
        return chart
    
    def labels_table(self):
        labels = pd.DataFrame([
            (self.x_start+(self.x_in/2),self.y_start, f"X in: {self.x_in}", "green"),
            (self.x_end*1.25,self.y_start+(self.y_in/2), f"Y out: {self.y_out}", "green"),
            ((self.x_end-self.x_start)/2+self.x_start,abs(self.y_end-self.y_start)/2, f"Execution price:\n{self.p_execution_truncated}", "green"),
        ], columns=['x','y','text', 'color'])
        return labels
    
    def view_labels(self):
        return self.labels_table().hvplot.labels(x='x',y='y',text='text', cmap='color', text_baseline='bottom')
    
    def view_chart(self):
        chart = self.view_points() * self.view_curves() * self.view_labels()
        chart.opts(height=400, width=1000, title="Uniswap Math")
        return chart
    
    def view(self):
        return pn.Column(self, self.view_chart)

In [26]:
u = Uniswap()

u.view()

Column
    [0] Column(margin=5, name='Uniswap', width=300)
        [0] StaticText(value='<b>Uniswap</b>')
        [1] FloatSlider(end=400, name='L', start=0.05, value=73.5)
        [2] FloatSlider(end=302.7, name='X end', start=0.05, value=167)
        [3] FloatSlider(end=226, name='Y start', value=188)
    [1] ParamMethod(method, _pane=HoloViews)